In [68]:
# We'll start with our library imports...
from __future__ import print_function

import numpy as np                 # to use numpy arrays
import tensorflow as tf            # to specify and run computation graphs
import tensorflow_datasets as tfds # to load training data
import matplotlib.pyplot as plt    # to visualize data and draw plots
from tqdm import tqdm              # to track progress of loops

import random

In [69]:
# Prepares the CIFAR-10 test dataset for testing.
def prepare_test_cifar10(): 
    test_ds = tfds.load('cifar10', split='test')
    
    prep_test_ds = []
    for data in test_ds: 
        prep_test_ds.append(data)
        
    return prep_test_ds

In [70]:
# Prepares the balanced CIFAR-10 train dataset for training. 
# NOTE: Probably did it in the most inefficient way possible. 
def prepare_train_cifar10(): 
    # Load CIFAR-10 dataset
    train_ds, info = tfds.load('cifar10', split='train', with_info=True)

    # Get class names
    class_names = info.features['label'].names

    # Create lists to hold images and labels in order from class 0 to class 9
    ordered_images = [[] for _ in range(10)]
    ordered_labels = [[] for _ in range(10)]

    # Iterate through the dataset and sort images and labels
    for example in train_ds:
        image, label = example['image'], example['label']
        ordered_images[label].append(image)
        ordered_labels[label].append(example['label'])

    # Concatenate lists
    images = []
    labels = []
    for i in range(10):
        images.extend(ordered_images[i])
        labels.extend(ordered_labels[i])

    # Convert lists to TensorFlow tensors
    images = tf.convert_to_tensor(images)
    labels = tf.convert_to_tensor(labels)
    
    prep_train_ds = []
    for i in range(50000): 
        prep_train_ds.append({
            'id': i,
            'image': images[i],
            'label': labels[i]
        })
        
    return prep_train_ds

In [71]:
#Takes the balanced CIFAR-10 train dataset as a parameter and imbalances it 
def prepare_imb_train_cifar10(bal_dataset):
    prep_imb_train_ds = []
    balanced_dataset = bal_dataset
    
    #Remove a certain amount of data for each class in CIFAR-10. The amount of data removed is incremated by a rate of 0.1.
    for i in range(10): 
        prep_imb_train_ds.append(balanced_dataset[(5000*i):(5000 *(i+1))])

        number_remove = int(5000 * (0.1*i))
        del prep_imb_train_ds[i][:number_remove]
    
    prep_imb_train_ds = sum(prep_imb_train_ds, [])
    
    return prep_imb_train_ds

In [72]:
def shuffle_list(lst):
    random.shuffle(lst)
    return lst

In [73]:
test_cifar10_ds = prepare_test_cifar10()
train_cifar10_ds = prepare_train_cifar10()

In [74]:
imb_train_cifar10_ds = prepare_imb_train_cifar10(train_cifar10_ds)

In [75]:
######### MODEL ARCH #########
hidden_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.relu)
hidden_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.relu)
pool_1 = tf.keras.layers.MaxPool2D(padding='same')
hidden_3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation=tf.nn.relu)
hidden_4 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation=tf.nn.relu)
pool_2 = tf.keras.layers.MaxPool2D(padding='same')
flatten = tf.keras.layers.Flatten()
output = tf.keras.layers.Dense(10)
conv_classifier = tf.keras.Sequential([hidden_1, hidden_2, pool_1, hidden_3, hidden_4, pool_2, flatten, output])


optimizer = tf.keras.optimizers.Adam()

In [82]:
shuffle_list(test_cifar10_ds)
shuffle_list(train_cifar10_ds)

[{'id': 21599,
  'image': <tf.Tensor: shape=(32, 32, 3), dtype=uint8, numpy=
  array([[[238, 239, 234],
          [233, 234, 229],
          [231, 232, 227],
          ...,
          [ 29, 147, 174],
          [111, 153, 117],
          [150, 182, 105]],
  
         [[239, 240, 235],
          [233, 234, 229],
          [231, 232, 227],
          ...,
          [ 49, 139, 164],
          [140, 182,  78],
          [154, 197,  62]],
  
         [[239, 240, 235],
          [234, 235, 230],
          [232, 233, 228],
          ...,
          [ 63, 133, 111],
          [139, 186,  53],
          [155, 206,  91]],
  
         ...,
  
         [[146, 141,  70],
          [120, 151,  62],
          [161, 200,  80],
          ...,
          [152, 139,  79],
          [146, 145,  91],
          [168, 152, 102]],
  
         [[129, 105,  64],
          [126, 151,  76],
          [137, 173,  74],
          ...,
          [147, 134,  73],
          [153, 144,  92],
          [159, 141,  98]],
  
 

In [83]:
loss_values = []
accuracy_values = []
# Early Stopping Parameters
early_stopping_rounds = 5
best_loss = float('inf')
counter = 0
best_accuracy = float('inf')

for epoch in tqdm(range(5)):
    # for batch in tqdm(train):
    # print("Epoch:", epoch)
    for batch in train_cifar10_ds:
        with tf.GradientTape() as tape:
            # run network
            # x = tf.reshape(tf.cast(batch['image'], tf.float32)/255.0, [-1, 784])
            image = tf.cast(tf.expand_dims(batch['image'], axis=0), tf.float32)
            labels = batch['label']
            logits = conv_classifier(image)
            
            # print(tf.squeeze(logits, axis=0))
            # print(labels)
            
            # calculate loss
            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=tf.squeeze(logits, axis=0), labels=labels)
        loss_values.append(loss)
        
        # gradient update
        grads = tape.gradient(loss, conv_classifier.trainable_variables)
        optimizer.apply_gradients(zip(grads, conv_classifier.trainable_variables))
        
        # calculate accuracy
        predictions = tf.argmax(logits, axis=1)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))
        accuracy_values.append(accuracy)
        
        # Early stopping
        # if accuracy.numpy() < best_accuracy and np.mean(loss_values) < best_loss:
        #     best_accuracy = accuracy.numpy()
        #     best_loss = np.mean(loss_values)
        #     counter = 0
        # else:
        #     counter += 1
        # if counter >= early_stopping_rounds:
        #     # print("Early stopping!")
        #     break
            
# accuracy
print("Accuracy During Training:", np.mean(accuracy_values))

100%|██████████| 5/5 [23:56<00:00, 287.27s/it]


Accuracy During Training: 0.43162


In [85]:
loss_values_t = []
accuracy_values_t = []
# Early Stopping Parameters
early_stopping_rounds = 5
best_loss = float('inf')
counter = 0
best_accuracy = float('inf')

for batch in test_cifar10_ds:
    with tf.GradientTape() as tape:
        # run network
        # x = tf.reshape(tf.cast(batch['image'], tf.float32)/255.0, [-1, 784])
        image = tf.cast(tf.expand_dims(batch['image'], axis=0), tf.float32)
        labels = batch['label']
        logits = conv_classifier(image)

        # print(tf.squeeze(logits, axis=0))
        # print(labels)

        # calculate loss
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=tf.squeeze(logits, axis=0), labels=labels)
    loss_values_t.append(loss)

    # calculate accuracy
    predictions = tf.argmax(logits, axis=1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))
    accuracy_values_t.append(accuracy)

    # Early stopping
    if accuracy.numpy() < best_accuracy and np.mean(loss_values_t) < best_loss:
        best_accuracy = accuracy.numpy()
        best_loss = np.mean(loss_values_t)
        counter = 0
    else:
        counter += 1
    if counter >= early_stopping_rounds:
        print("Early stopping!")
        break
            
# accuracy
print("Accuracy During Training:", np.mean(accuracy_values_t))

Early stopping!
Accuracy During Training: 0.8333333
